# Bert QA サンプル

https://towardsdatascience.com/question-answering-with-a-fine-tuned-bert-bc4dafd45626

## ライブラリ導入

In [ ]:
# transformersの導入
!pip install transformers | tail -n 1

In [ ]:
# ライブラリのインポート
import pandas as pd
import numpy as np
from IPython.display import display

import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

## テキストデータ準備

### coqa データ読み込み

https://stanfordnlp.github.io/coqa/

CoQAは、質問応答システムを構築するための大規模なデータセットです。CoQAチャレンジの目標は、テキストのパッセージを理解し、会話に現れる一連の相互に関連する質問に答える機械学習モデルの性能を測定することです。

In [ ]:
# 訓練用データ読み込み
coqa = pd.read_json('http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json')
# データの確認
display(coqa.head())

### 読み込みデータ確認

In [ ]:
# データの先頭行の内容表示
item = coqa.loc[0,'data']
print(item)

In [ ]:
# キーの確認
print(item.keys())

In [ ]:
# storyの表示
print(item['story'])

In [ ]:
# questionsの表示
for text in item['questions']:
    print(text)

In [ ]:
# answersの表示
for text in item['answers']:
    print(text)

### データ加工

In [ ]:
#　テキスト(text)、質問(question)、回答(answer)の抽出
# 一つのテキストに対して質問、回答のペアは複数対応
cols = ["text","question","answer"]

# 抽出リストの１行分
comp_list = []
for index, row in coqa.iterrows():

    # 質問の個数だけ繰り返し
    for i in range(len(row["data"]["questions"])):
        temp_list = []

        # text
        temp_list.append(row["data"]["story"])

        # i番目の質問
        temp_list.append(row["data"]["questions"][i]["input_text"])

        # i番目の回答
        temp_list.append(row["data"]["answers"][i]["input_text"])

        # リストのリストを生成
        comp_list.append(temp_list)

# comp_listからデータフレームを生成
data = pd.DataFrame(comp_list, columns=cols) 

#　２度目以降のために、csvファイルとしても保存
data.to_csv("CoQA_data.csv", index=False)

### 加工後データ確認

In [ ]:
# 先頭と、最後の内容表示
display(data.head())
display(data.tail())

In [ ]:
# 10番目の要素ののtext, question, answerの確認
index = 10
print(f'Text: \n{data.loc[index].text}\n' )
print(f'Question: {data.loc[index].question}\n' )
print(f'Answer: {data.loc[index].answer}\n')

In [ ]:
# データの総数
print("Number of question and answers: ", len(data))

## BERT QAモデル読み込み
今回利用するモデルは、QA形式のファインチューニングも済んでいて、入力データを渡すだけで結果を返すことが可能

In [ ]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

## BERTによる予測

### 特定の１セット取得

In [ ]:
# 特定の１セットを抽出
index = 10
text = data["text"][index]
question = data["question"][index]
answer = data["answer"][index]

### 入力データのエンコード

In [ ]:
# 質問、テキストの組をエンコードする
input_ids = tokenizer.encode(question, text)

# エンコードの値から逆向きにトークンの一覧を取得
tokens = tokenizer.convert_ids_to_tokens(input_ids)

### エンコード結果の確認

In [ ]:
# input_idsの長さを計算
input_len = len(input_ids)
print(f'入力文字列の長さ: {input_len}')

In [ ]:
# input_idsの先頭２０要素
print(input_ids[:20])

# tokensの先頭20要素
print(tokens[:20])

# 先頭20要素をinput_ids, tokensと対応付けて表示
for token, id in zip(tokens[:20], input_ids[:20]):
    print('{:8}{:8,}'.format(token,id))

### segment_idsの計算
QAモデルでは入力として、インデックス化したテキストだけでなく、segmentベクトルを引数で渡す必要がある。  
以下の実装でその準備を行う。

In [ ]:
#　[SEP] tokenの最初の位置
sep_idx = input_ids.index(tokenizer.sep_token_id)
print("SEP token index: ", sep_idx)

In [ ]:
#　セグメントAのトークン数　
# (pythonのindexはゼロから始まるので、sep token indexより１大きい)
num_seg_a = sep_idx+1
print("Number of tokens in segment A: ", num_seg_a)

In [ ]:
# セグメントBのトークン数
num_seg_b = len(input_ids) - num_seg_a
print("Number of tokens in segment B: ", num_seg_b)

In [ ]:
# segment_idsの計算
segment_ids = [0]*num_seg_a + [1]*num_seg_b
print(segment_ids)

In [ ]:
#　segmeind_idsの長さとinput_idsの長さが一致していることの確認
assert len(segment_ids) == len(input_ids)

### 予測の実施

In [ ]:
# inpit_idsとsegment_idsを用いて予測の実施
output = model(torch.tensor([input_ids]),  token_type_ids=torch.tensor([segment_ids]))

In [ ]:
# 結果の確認
print(output)

In [ ]:
# answer_start と answer_endの計算
answer_start = torch.argmax(output.start_logits)
answer_end = torch.argmax(output.end_logits)
if answer_end >= answer_start:
    answer = " ".join(tokens[answer_start:answer_end+1])
else:
    answer = ("I am unable to find the answer to this question. Can you please ask another question?")

#  結果の確認
print(answer_start, answer_end, answer)


In [ ]:
question_cap = question.capitalize()
answer_cap = answer.capitalize()

print(f"\nQuestion:\n{question_cap}")
print(f"\nAnswer:\n{answer_cap}")